<a href="https://colab.research.google.com/github/zboraon/LakeBafeClosure_Bayesian_cps/blob/main/Bayesian_changepoint_Bafa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary libraries



Install and load the necessary libraries.


In [ ]:
system("apt install -y jags")
system("apt install -y r-base")

In [ ]:
list.of.packages <- c("runjags", "RCurl","signal", "dplyr", "zoo")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, dependencies = TRUE)

In [ ]:
rm(list = ls())

In [ ]:
library("signal")
library("dplyr")
library("zoo")
library("runjags")
library("RCurl")
library("coda")

# Utilities from Kruschke

In [ ]:
#------------------------------------------------------------------------------
# Check that required packages are installed:
want = c("parallel","rjags","runjags","compute.es")
have = want %in% rownames(installed.packages())
if ( any(!have) ) { install.packages( want[!have] ) }

# Load rjags. Assumes JAGS is already installed.
try( library(rjags) )
# Load runjags. Assumes JAGS is already installed.
try( library(runjags) )
try( runjags.options( inits.warning=FALSE , rng.warning=FALSE ) )

# set default number of chains and parallelness for MCMC:
library(parallel) # for detectCores().
nCores = detectCores() 
if ( !is.finite(nCores) ) { nCores = 1 } 
if ( nCores > 4 ) { 
  nChainsDefault = 4  # because JAGS has only 4 rng's.
  runjagsMethodDefault = "parallel"
}
if ( nCores == 4 ) { 
  nChainsDefault = 3  # save 1 core for other processes.
  runjagsMethodDefault = "parallel"
}
if ( nCores < 4 ) { 
  nChainsDefault = 3 
  runjagsMethodDefault = "rjags" # NOT parallel
}

#------------------------------------------------------------------------------
# Functions for opening and saving graphics that operate the same for 
# Windows and Macintosh and Linux operating systems. At least, that's the hope!

openGraph = function( width=7 , height=7 , mag=1.0 , ... ) {
  if ( .Platform$OS.type != "windows" ) { # Mac OS, Linux
    tryInfo = try( X11( width=width*mag , height=height*mag , type="cairo" , 
                        ... ) )
    if ( class(tryInfo)=="try-error" ) {
      lineInput = readline("WARNING: Previous graphics windows will be closed because of too many open windows.
      \nTO CONTINUE, PRESS <ENTER> IN R CONSOLE.\n")
      graphics.off() 
      X11( width=width*mag , height=height*mag , type="cairo" , ... )
    }
  } else { # Windows OS
    tryInfo = try( windows( width=width*mag , height=height*mag , ... ) )
    if ( class(tryInfo)=="try-error" ) {
      lineInput = readline("WARNING: Previous graphics windows will be closed because of too many open windows.
      \nTO CONTINUE, PRESS <ENTER> IN R CONSOLE.\n")
      graphics.off() 
      windows( width=width*mag , height=height*mag , ... )    
    }
  }
}

saveGraph = function( file="saveGraphOutput" , type="pdf" , ... ) {
  if ( .Platform$OS.type != "windows" ) { # Mac OS, Linux
    if ( any( type == c("png","jpeg","jpg","tiff","bmp")) ) {
      sptype = type
      if ( type == "jpg" ) { sptype = "jpeg" }
      savePlot( file=paste0(file,".",type) , type=sptype , ... )     
    }
    if ( type == "pdf" ) {
      dev.copy2pdf(file=paste0(file,".",type) , ... )
    }
    if ( type == "eps" ) {
      dev.copy2eps(file=paste0(file,".",type) , ... )
    }
  } else { # Windows OS
    file=paste0(file,".",type) 
    savePlot( file=file , type=type , ... )
  }
}

#------------------------------------------------------------------------------
# Functions for computing limits of HDI's:

HDIofMCMC = function( sampleVec , credMass=0.89 ) {
  # Computes highest density interval from a sample of representative values,
  #   estimated as shortest credible interval.
  # Arguments:
  #   sampleVec
  #     is a vector of representative values from a probability distribution.
  #   credMass
  #     is a scalar between 0 and 1, indicating the mass within the credible
  #     interval that is to be estimated.
  # Value:
  #   HDIlim is a vector containing the limits of the HDI
  sortedPts = sort( sampleVec )
  ciIdxInc = ceiling( credMass * length( sortedPts ) )
  nCIs = length( sortedPts ) - ciIdxInc
  ciWidth = rep( 0 , nCIs )
  for ( i in 1:nCIs ) {
    ciWidth[ i ] = sortedPts[ i + ciIdxInc ] - sortedPts[ i ]
  }
  HDImin = sortedPts[ which.min( ciWidth ) ]
  HDImax = sortedPts[ which.min( ciWidth ) + ciIdxInc ]
  HDIlim = c( HDImin , HDImax )
  return( HDIlim )
}

HDIofICDF = function( ICDFname , credMass=0.89 , tol=1e-8 , ... ) {
  # Arguments:
  #   ICDFname is R's name for the inverse cumulative density function
  #     of the distribution.
  #   credMass is the desired mass of the HDI region.
  #   tol is passed to R's optimize function.
  # Return value:
  #   Highest density iterval (HDI) limits in a vector.
  # Example of use: For determining HDI of a beta(30,12) distribution, type
  #   HDIofICDF( qbeta , shape1 = 30 , shape2 = 12 )
  #   Notice that the parameters of the ICDFname must be explicitly named;
  #   e.g., HDIofICDF( qbeta , 30 , 12 ) does not work.
  # Adapted and corrected from Greg Snow's TeachingDemos package.
  incredMass =  1.0 - credMass
  intervalWidth = function( lowTailPr , ICDFname , credMass , ... ) {
    ICDFname( credMass + lowTailPr , ... ) - ICDFname( lowTailPr , ... )
  }
  optInfo = optimize( intervalWidth , c( 0 , incredMass ) , ICDFname=ICDFname ,
                      credMass=credMass , tol=tol , ... )
  HDIlowTailPr = optInfo$minimum
  return( c( ICDFname( HDIlowTailPr , ... ) ,
             ICDFname( credMass + HDIlowTailPr , ... ) ) )
}

HDIofGrid = function( probMassVec , credMass=0.89 ) {
  # Arguments:
  #   probMassVec is a vector of probability masses at each grid point.
  #   credMass is the desired mass of the HDI region.
  # Return value:
  #   A list with components:
  #   indices is a vector of indices that are in the HDI
  #   mass is the total mass of the included indices
  #   height is the smallest component probability mass in the HDI
  # Example of use: For determining HDI of a beta(30,12) distribution
  #   approximated on a grid:
  #   > probDensityVec = dbeta( seq(0,1,length=201) , 30 , 12 )
  #   > probMassVec = probDensityVec / sum( probDensityVec )
  #   > HDIinfo = HDIofGrid( probMassVec )
  #   > show( HDIinfo )
  sortedProbMass = sort( probMassVec , decreasing=TRUE )
  HDIheightIdx = min( which( cumsum( sortedProbMass ) >= credMass ) )
  HDIheight = sortedProbMass[ HDIheightIdx ]
  HDImass = sum( probMassVec[ probMassVec >= HDIheight ] )
  return( list( indices = which( probMassVec >= HDIheight ) ,
                mass = HDImass , height = HDIheight ) )
}

#------------------------------------------------------------------------------
# Function(s) for plotting properties of mcmc coda objects.

DbdaAcfPlot = function( codaObject , parName=varnames(codaObject)[1] , plColors=NULL ) {
  if ( all( parName != varnames(codaObject) ) ) { 
    stop("parName must be a column name of coda object")
  }
  nChain = length(codaObject)
  if ( is.null(plColors) ) plColors=1:nChain
  xMat = NULL
  yMat = NULL
  for ( cIdx in 1:nChain ) {
    acfInfo = acf(codaObject[,c(parName)][[cIdx]],plot=FALSE) 
    xMat = cbind(xMat,acfInfo$lag)
    yMat = cbind(yMat,acfInfo$acf)
  }
  matplot( xMat , yMat , type="o" , pch=20 , col=plColors , ylim=c(0,1) ,
           main="" , xlab="Lag" , ylab="Autocorrelation" )
  abline(h=0,lty="dashed")
  EffChnLngth = effectiveSize(codaObject[,c(parName)])
  text( x=max(xMat) , y=max(yMat) , adj=c(1.0,1.0) , cex=1.25 ,
        labels=paste("ESS =",round(EffChnLngth,1)) )
}

DbdaDensPlot = function( codaObject , parName=varnames(codaObject)[1] , plColors=NULL ) {
  if ( all( parName != varnames(codaObject) ) ) { 
    stop("parName must be a column name of coda object")
  }
  nChain = length(codaObject) # or nchain(codaObject)
  if ( is.null(plColors) ) plColors=1:nChain
  xMat = NULL
  yMat = NULL
  hdiLims = NULL
  for ( cIdx in 1:nChain ) {
    densInfo = density(codaObject[,c(parName)][[cIdx]]) 
    xMat = cbind(xMat,densInfo$x)
    yMat = cbind(yMat,densInfo$y)
    hdiLims = cbind(hdiLims,HDIofMCMC(codaObject[,c(parName)][[cIdx]]))
  }
  matplot( xMat , yMat , type="l" , col=plColors , 
           main="" , xlab="Param. Value" , ylab="Density" )
  abline(h=0)
  points( hdiLims[1,] , rep(0,nChain) , col=plColors , pch="|" )
  points( hdiLims[2,] , rep(0,nChain) , col=plColors , pch="|" )
  text( mean(hdiLims) , 0 , "89% HDI" , adj=c(0.5,-0.2) )
  EffChnLngth = effectiveSize(codaObject[,c(parName)])
  MCSE = sd(as.matrix(codaObject[,c(parName)]))/sqrt(EffChnLngth) 
  text( max(xMat) , max(yMat) , adj=c(1.0,1.0) , cex=1.25 ,
        paste("MCSE =\n",signif(MCSE,3)) )
}

diagMCMC = function( codaObject , parName=varnames(codaObject)[1] ,
                     saveName=NULL , saveType="jpg" ) {
  DBDAplColors = c("skyblue","black","royalblue","steelblue")
  openGraph(height=5,width=7)
  par( mar=0.5+c(3,4,1,0) , oma=0.1+c(0,0,2,0) , mgp=c(2.25,0.7,0) , 
       cex.lab=1.5 )
  layout(matrix(1:4,nrow=2))
  # traceplot and gelman.plot are from CODA package:
  require(coda)
  coda::traceplot( codaObject[,c(parName)] , main="" , ylab="Param. Value" ,
                   col=DBDAplColors ) 
  tryVal = try(
    coda::gelman.plot( codaObject[,c(parName)] , main="" , auto.layout=FALSE , 
                       col=DBDAplColors )
  )  
  # if it runs, gelman.plot returns a list with finite shrink values:
  if ( class(tryVal)=="try-error" ) {
    plot.new() 
    print(paste0("Warning: coda::gelman.plot fails for ",parName))
  } else { 
    if ( class(tryVal)=="list" & !is.finite(tryVal$shrink[1]) ) {
      plot.new() 
      print(paste0("Warning: coda::gelman.plot fails for ",parName))
    }
  }
  DbdaAcfPlot(codaObject,parName,plColors=DBDAplColors)
  DbdaDensPlot(codaObject,parName,plColors=DBDAplColors)
  mtext( text=parName , outer=TRUE , adj=c(0.5,0.5) , cex=2.0 )
  if ( !is.null(saveName) ) {
    saveGraph( file=paste0(saveName, "_", parName,"_Diag_dt"), type=saveType)
  }
}

diagStanFit = function( stanFit , parName ,
                        saveName=NULL , saveType="jpg" ) {
  codaFit = mcmc.list( lapply( 1:ncol(stanFit) , 
                               function(x) { mcmc(as.array(stanFit)[,x,]) } ) )
  DBDAplColors = c("skyblue","black","royalblue","steelblue")
  openGraph(height=5,width=7)
  par( mar=0.5+c(3,4,1,0) , oma=0.1+c(0,0,2,0) , mgp=c(2.25,0.7,0) , cex.lab=1.5 )
  layout(matrix(1:4,nrow=2))
  # traceplot is from rstan package
  require(rstan)
  traceplot(stanFit,pars=parName,nrow=1,ncol=1)#,main="",ylab="Param. Value",col=DBDAplColors) 
  # gelman.plot are from CODA package:
  require(coda)
  tryVal = try(
    coda::gelman.plot( codaObject[,c(parName)] , main="" , auto.layout=FALSE , 
                       col=DBDAplColors )
  )
  # if it runs, gelman.plot returns a list with finite shrink values:
  if ( class(tryVal)=="try-error" ) {
    plot.new() 
    print(paste0("Warning: coda::gelman.plot fails for ",parName))
  } else { 
    if ( class(tryVal)=="list" & !is.finite(tryVal$shrink[1]) ) {
      plot.new() 
      print(paste0("Warning: coda::gelman.plot fails for ",parName))
    }
  }
  DbdaAcfPlot(codaFit,parName,plColors=DBDAplColors)
  DbdaDensPlot(codaFit,parName,plColors=DBDAplColors)
  mtext( text=parName , outer=TRUE , adj=c(0.5,0.5) , cex=2.0 )
  if ( !is.null(saveName) ) {
    saveGraph( file=paste0(saveName,"Diag",parName), type=saveType)
  }
}

#------------------------------------------------------------------------------
# Functions for summarizing and plotting distribution of a large sample; 
# typically applied to MCMC posterior.

normalize = function( v ){ return( v / sum(v) ) }

require(coda) # loaded by rjags, but redundancy doesn't hurt

summarizePost = function( paramSampleVec , 
                          compVal=NULL , ROPE=NULL , credMass=0.89 ) {
  meanParam = mean( paramSampleVec )
  medianParam = median( paramSampleVec )
  dres = density( paramSampleVec )
  modeParam = dres$x[which.max(dres$y)]
  mcmcEffSz = round( effectiveSize( paramSampleVec ) , 1 )
  names(mcmcEffSz) = NULL
  hdiLim = HDIofMCMC( paramSampleVec , credMass=credMass )
  if ( !is.null(compVal) ) {
    pcgtCompVal = ( 100 * sum( paramSampleVec > compVal ) 
                    / length( paramSampleVec ) )
  } else {
    compVal=NA
    pcgtCompVal=NA
  }
  if ( !is.null(ROPE) ) {
    pcltRope = ( 100 * sum( paramSampleVec < ROPE[1] ) 
                 / length( paramSampleVec ) )
    pcgtRope = ( 100 * sum( paramSampleVec > ROPE[2] ) 
                 / length( paramSampleVec ) )
    pcinRope = 100-(pcltRope+pcgtRope)
  } else { 
    ROPE = c(NA,NA)
    pcltRope=NA 
    pcgtRope=NA 
    pcinRope=NA 
  }  
  return( c( Mean=meanParam , Median=medianParam , Mode=modeParam , 
             ESS=mcmcEffSz ,
             HDImass=credMass , HDIlow=hdiLim[1] , HDIhigh=hdiLim[2] , 
             CompVal=compVal , PcntGtCompVal=pcgtCompVal , 
             ROPElow=ROPE[1] , ROPEhigh=ROPE[2] ,
             PcntLtROPE=pcltRope , PcntInROPE=pcinRope , PcntGtROPE=pcgtRope ) )
}

plotPost = function( paramSampleVec , cenTend=c("mode","median","mean")[2] , 
                     compVal=NULL, ROPE=NULL, credMass=0.89, HDItextPlace=0.7, 
                     xlab=NULL , xlim=NULL , yaxt=NULL , ylab=NULL , 
                     main=NULL , cex=NULL , cex.lab=NULL ,
                     col=NULL , border=NULL , showCurve=FALSE , breaks=NULL , 
                     ... ) {
  # Override defaults of hist function, if not specified by user:
  # (additional arguments "..." are passed to the hist function)
  if ( is.null(xlab) ) xlab="Param. Val."
  if ( is.null(cex.lab) ) cex.lab=1.5
  if ( is.null(cex) ) cex=1.4
  if ( is.null(xlim) ) xlim=range( c( compVal , ROPE , paramSampleVec ) )
  if ( is.null(main) ) main=""
  if ( is.null(yaxt) ) yaxt="n"
  if ( is.null(ylab) ) ylab=""
  if ( is.null(col) ) col="skyblue"
  if ( is.null(border) ) border="white"
  
  # convert coda object to matrix:
  if ( class(paramSampleVec) == "mcmc.list" ) {
    paramSampleVec = as.matrix(paramSampleVec)
  }
  
  summaryColNames = c("ESS","mean","median","mode",
                      "hdiMass","hdiLow","hdiHigh",
                      "compVal","pGtCompVal",
                      "ROPElow","ROPEhigh","pLtROPE","pInROPE","pGtROPE")
  postSummary = matrix( NA , nrow=1 , ncol=length(summaryColNames) , 
                        dimnames=list( c( xlab ) , summaryColNames ) )
  
  # require(coda) # for effectiveSize function
  postSummary[,"ESS"] = effectiveSize(paramSampleVec)
  
  postSummary[,"mean"] = mean(paramSampleVec)
  postSummary[,"median"] = median(paramSampleVec)
  mcmcDensity = density(paramSampleVec)
  postSummary[,"mode"] = mcmcDensity$x[which.max(mcmcDensity$y)]
  
  HDI = HDIofMCMC( paramSampleVec , credMass )
  postSummary[,"hdiMass"]=credMass
  postSummary[,"hdiLow"]=HDI[1]
  postSummary[,"hdiHigh"]=HDI[2]
  
  # Plot histogram.
  cvCol = "darkgreen"
  ropeCol = "darkred"
  if ( is.null(breaks) ) {
    if ( max(paramSampleVec) > min(paramSampleVec) ) {
      breaks = c( seq( from=min(paramSampleVec) , to=max(paramSampleVec) ,
                       by=(HDI[2]-HDI[1])/18 ) , max(paramSampleVec) )
    } else {
      breaks=c(min(paramSampleVec)-1.0E-6,max(paramSampleVec)+1.0E-6)
      border="skyblue"
    }
  }
  if ( !showCurve ) {
    par(xpd=NA)
    histinfo = hist( paramSampleVec , xlab=xlab , yaxt=yaxt , ylab=ylab ,
                     freq=F , border=border , col=col ,
                     xlim=xlim , main=main , cex=cex , cex.lab=cex.lab ,
                     breaks=breaks , ... )
  }
  if ( showCurve ) {
    par(xpd=NA)
    histinfo = hist( paramSampleVec , plot=F )
    densCurve = density( paramSampleVec , adjust=2 )
    plot( densCurve$x , densCurve$y , type="l" , lwd=5 , col=col , bty="n" ,
          xlim=xlim , xlab=xlab , yaxt=yaxt , ylab=ylab ,
          main=main , cex=cex , cex.lab=cex.lab , ... )
  }
  cenTendHt = 0.9*max(histinfo$density)
  cvHt = 0.7*max(histinfo$density)
  ROPEtextHt = 0.55*max(histinfo$density)
  # Display central tendency:
  mn = mean(paramSampleVec)
  med = median(paramSampleVec)
  mcmcDensity = density(paramSampleVec)
  mo = mcmcDensity$x[which.max(mcmcDensity$y)]
  if ( cenTend=="mode" ){ 
    text( mo , cenTendHt ,
          bquote(mode==.(signif(mo,3))) , adj=c(.5,0) , cex=cex )
  }
  if ( cenTend=="median" ){ 
    text( med , cenTendHt ,
          bquote(median==.(signif(med,3))) , adj=c(.5,0) , cex=cex , col=cvCol )
  }
  if ( cenTend=="mean" ){ 
    text( mn , cenTendHt ,
          bquote(mean==.(signif(mn,3))) , adj=c(.5,0) , cex=cex )
  }
  # Display the comparison value.
  if ( !is.null( compVal ) ) {
    pGtCompVal = sum( paramSampleVec > compVal ) / length( paramSampleVec ) 
    pLtCompVal = 1 - pGtCompVal
    lines( c(compVal,compVal) , c(0.96*cvHt,0) , 
           lty="dotted" , lwd=2 , col=cvCol )
    text( compVal , cvHt ,
          bquote( .(round(100*pLtCompVal,1)) * "% < " *
                   .(signif(compVal,3)) * " < " * 
                   .(round(100*pGtCompVal,1)) * "%" ) ,
          adj=c(pLtCompVal,0) , cex=0.8*cex , col=cvCol )
    postSummary[,"compVal"] = compVal
    postSummary[,"pGtCompVal"] = pGtCompVal
  }
  # Display the ROPE.
  if ( !is.null( ROPE ) ) {
    pInROPE = ( sum( paramSampleVec > ROPE[1] & paramSampleVec < ROPE[2] )
                / length( paramSampleVec ) )
    pGtROPE = ( sum( paramSampleVec >= ROPE[2] ) / length( paramSampleVec ) )
    pLtROPE = ( sum( paramSampleVec <= ROPE[1] ) / length( paramSampleVec ) )
    lines( c(ROPE[1],ROPE[1]) , c(0.96*ROPEtextHt,0) , lty="dotted" , lwd=2 ,
           col=ropeCol )
    lines( c(ROPE[2],ROPE[2]) , c(0.96*ROPEtextHt,0) , lty="dotted" , lwd=2 ,
           col=ropeCol)
    text( mean(ROPE) , ROPEtextHt ,
          bquote( .(round(100*pLtROPE,1)) * "% < " * .(ROPE[1]) * " < " * 
                   .(round(100*pInROPE,1)) * "% < " * .(ROPE[2]) * " < " * 
                   .(round(100*pGtROPE,1)) * "%" ) ,
          adj=c(pLtROPE+.5*pInROPE,0) , cex=1 , col=ropeCol )
    
    postSummary[,"ROPElow"]=ROPE[1] 
    postSummary[,"ROPEhigh"]=ROPE[2] 
    postSummary[,"pLtROPE"]=pLtROPE
    postSummary[,"pInROPE"]=pInROPE
    postSummary[,"pGtROPE"]=pGtROPE
  }
  # Display the HDI.
  lines( HDI , c(0,0) , lwd=4 , lend=1 )
  text( mean(HDI) , 0 , bquote(.(100*credMass) * "% HDI" ) ,
        adj=c(.5,-1.7) , cex=cex )
  text( HDI[1] , 0 , bquote(.(signif(HDI[1],3))) ,
        adj=c(HDItextPlace,-0.5) , cex=cex )
  text( HDI[2] , 0 , bquote(.(signif(HDI[2],3))) ,
        adj=c(1.0-HDItextPlace,-0.5) , cex=cex )
  par(xpd=F)
  #
  return( postSummary )
}

#------------------------------------------------------------------------------

# Shape parameters from central tendency and scale:

betaABfromMeanKappa = function( mean , kappa ) {
  if ( mean <=0 | mean >= 1) stop("must have 0 < mean < 1")
  if ( kappa <=0 ) stop("kappa must be > 0")
  a = mean * kappa
  b = ( 1.0 - mean ) * kappa
  return( list( a=a , b=b ) )
}

betaABfromModeKappa = function( mode , kappa ) {
  if ( mode <=0 | mode >= 1) stop("must have 0 < mode < 1")
  if ( kappa <=2 ) stop("kappa must be > 2 for mode parameterization")
  a = mode * ( kappa - 2 ) + 1
  b = ( 1.0 - mode ) * ( kappa - 2 ) + 1
  return( list( a=a , b=b ) )
}

betaABfromMeanSD = function( mean , sd ) {
  if ( mean <=0 | mean >= 1) stop("must have 0 < mean < 1")
  if ( sd <= 0 ) stop("sd must be > 0")
  kappa = mean*(1-mean)/sd^2 - 1
  if ( kappa <= 0 ) stop("invalid combination of mean and sd")
  a = mean * kappa
  b = ( 1.0 - mean ) * kappa
  return( list( a=a , b=b ) )
}

gammaShRaFromMeanSD = function( mean , sd ) {
  if ( mean <=0 ) stop("mean must be > 0")
  if ( sd <=0 ) stop("sd must be > 0")
  shape = mean^2/sd^2
  rate = mean/sd^2
  return( list( shape=shape , rate=rate ) )
}

gammaShRaFromModeSD = function( mode , sd ) {
  if ( mode <=0 ) stop("mode must be > 0")
  if ( sd <=0 ) stop("sd must be > 0")
  rate = ( mode + sqrt( mode^2 + 4 * sd^2 ) ) / ( 2 * sd^2 )
  shape = 1 + mode * rate
  return( list( shape=shape , rate=rate ) )
}

#------------------------------------------------------------------------------

# Make some data files for examples...
createDataFiles=FALSE
if ( createDataFiles ) {
  
  source("HtWtDataGenerator.R")
  N=300
  m = HtWtDataGenerator( N , rndsd=47405 )
  write.csv( file=paste0("HtWtData",N,".csv") , row.names=FALSE , m )
  
  
  # Function for generating normal data with normal outliers:
  genYwithOut = function( N , pcntOut=15 , sdOut=3.0 ) {
    inl = rnorm( N-ceiling(pcntOut/100*N) )
    out = rnorm(   ceiling(pcntOut/100*N) )
    inl = (inl-mean(inl))/sd(inl)
    out = (out-mean(out))/sd(out) * sdOut
    return(c(inl,out))
  }
  
  # Two-group IQ scores with outliers 
  set.seed(47405)
  y1 = round(pmax(50,genYwithOut(63,20,3.5)*17.5+106))
  y2 = round(pmax(50,genYwithOut(57,20,3.5)*10+100))
  write.csv( file="TwoGroupIQ.csv" , row.names=FALSE ,
             data.frame( Score=c(y1,y2) , 
                         Group=c(rep("Smart Drug",length(y1)),
                                 rep("Placebo",length(y2))) ) )
  
  # One-group log-normal
  set.seed(47405)
  z = rnorm(123)
  logY = (z-mean(z))/sd(z) * 0.5 + 5.5 # logY has mean 5.5 and sd 0.5
  y = round( exp(logY) , 2 )
  write.csv( file="OneGroupLogNormal.csv" , row.names=FALSE ,
             cbind(y) )
  
  # One-group gamma
  desiredMode = 250
  desiredSD = 100
  desiredRate = (desiredMode+sqrt(desiredMode^2+4*desiredSD^2))/(2*desiredSD^2)
  desiredShape = 1+desiredMode*desiredRate
  set.seed(47405)
  y = round( rgamma( 153 , shape=desiredShape , rate=desiredRate ) , 2 )
  write.csv( file="OneGroupGamma.csv" , row.names=FALSE , cbind(y) )
  
} # end if createDataFiles


# Functions


In [ ]:
changepnt_lakebafa = function(testeddata, inits = NA,
                            sample = 10000, adaptSteps = 4000, 
                            burnInSteps = 2000, nChains = 4, 
                            numSavedSteps = 10000, thinSteps = 1){

 # parameters to watch 
jags_pars <- c("betaforamO18", "betaforamC13", "betaostraO18", 
               "betaostraC13", "betaTOC",
               "alphaforamO18", "alphaforamC13", "alphaostraO18", 
               "alphaostraC13", "alphaTOC",
                "tchangeforamO18", "tchangeforamC13", "tchangeostraO18",
                "tchangeostraC13", "tchangeTOC",
                "t.change"
                ) 
  
model <- "
  # Standardize the data:
data {
  tm <- mean(agemean)
  terrm <- mean(ageerror)
  yforamO18m <- mean(yforamO18)
  yforamC13m <- mean(yforamC13)
  yostraO18m <- mean(yostraO18)
  yostraC13m <- mean(yostraC13)
  yTOCm      <- mean(yTOC)
  
  
  tsd <- sd(agemean)
  terrsd <- sd(ageerror)
  yforamO18sd <- sd(yforamO18)
  yforamC13sd <- sd(yforamC13)
  yostraO18sd <- sd(yostraO18)
  yostraC13sd <- sd(yostraC13)
  yTOCsd <- sd(yTOC)
  
  
  for (i in 1:L) {
    zt[i] <- (agemean[i] - tm) / tsd
    zterr[i] <- (ageerror[i] - terrm) / terrsd
    zyforamO18[i] <- (yforamO18[i] - yforamO18m) / yforamO18sd
    zyforamC13[i] <- (yforamC13[i] - yforamC13m) / yforamC13sd
    zyostraO18[i] <- (yostraO18[i] - yostraO18m) / yostraO18sd
    zyostraC13[i] <- (yostraC13[i] - yostraC13m) / yostraC13sd
    zyTOC[i] <- (yTOC[i] - yTOCm) / yTOCsd
  }
  zt_min <- min(zt)
  zt_max <- max(zt)

  zcp1_priormean <- ( 520 - tm ) / tsd
  zcp2_priormean <- ( 2530 - tm ) / tsd
  zcp_priorsd <- ( 200 ) / tsd

}
model{
  # Likelihood:
  for(i in 1:L) {

    XforamO18_1[i] = min(zagemodel[i], ztchangeforamO18[1])
    XforamO18_2[i] = min(zagemodel[i], ztchangeforamO18[2]) - ztchangeforamO18[1]
    XforamO18_3[i] = min(zagemodel[i], zt_max) - ztchangeforamO18[2]

    zyforamO18[i]  ~  dt(zmuforamO18[i],  1/zsigmaforamO18[i]^2, nuforamO18) 
    zmuforamO18[i] <- 
      
      # Segment 1: 
      (zt[i] >= zt_min) * (zt[i] < ztchangeforamO18[1]) * zalphaforamO18[1] + 
    
      # Segment 2: 
      (zt[i] >= ztchangeforamO18[1]) * (zt[i] < ztchangeforamO18[2]) * zalphaforamO18[2] + 
    
      # Segment 3: 
      (zt[i] >= ztchangeforamO18[2]) * zalphaforamO18[3] 

    XforamC13_1[i] = min(zagemodel[i], ztchangeforamC13[1])
    XforamC13_2[i] = min(zagemodel[i], ztchangeforamC13[2]) - ztchangeforamC13[1]
    XforamC13_3[i] = min(zagemodel[i], zt_max) - ztchangeforamC13[2]

    zyforamC13[i]  ~  dt(zmuforamC13[i],  1/zsigmaforamC13[i]^2, nuforamC13) 
    zmuforamC13[i] <- 
      
      # Segment 1:
      (zt[i] >= zt_min) * (zt[i] < ztchangeforamC13[1]) * zalphaforamC13[1] + 
      
      # Segment 2: 
      (zt[i] >= ztchangeforamC13[1]) * (zt[i] < ztchangeforamC13[2]) * zalphaforamC13[2] + 
      
      # Segment 3: 
      (zt[i] >= ztchangeforamC13[2]) * zalphaforamC13[3] 
      
    XostraO18_1[i] = min(zagemodel[i], ztchangeostraO18[1])
    XostraO18_2[i] = min(zagemodel[i], ztchangeostraO18[2]) - ztchangeostraO18[1]
    XostraO18_3[i] = min(zagemodel[i], zt_max) - ztchangeostraO18[2]  
    
    zyostraO18[i]  ~  dt(zmuostraO18[i],  1/zsigmaostraO18[i]^2, nuostraO18) 
    zmuostraO18[i] <- 
      
      # Segment 1:
      (zt[i] >= zt_min) * (zt[i] < ztchangeostraO18[1]) * zalphaostraO18[1] + 
         
      # Segment 2: 
      (zt[i] >= ztchangeostraO18[1]) * (zt[i] < ztchangeostraO18[2]) * zalphaostraO18[2] + 
      (zt[i] >= ztchangeostraO18[1]) * (zt[i] < ztchangeostraO18[2]) * zbetaostraO18[2]  * XostraO18_2[i] + 
    
      # Segment 3: 
       (zt[i] >= ztchangeostraO18[2]) * zalphaostraO18[3] 

    XostraC13_1[i] = min(zagemodel[i], ztchangeostraC13[2])  
    XostraC13_2[i] = min(zagemodel[i], zt_max) - ztchangeostraC13[2]

    zyostraC13[i]  ~  dt(zmuostraC13[i],  1/zsigmaostraC13[i]^2, nuostraC13) 
    zmuostraC13[i] <- 
      # Segment 1: 
       (zt[i] >= zt_min) * (zt[i] < ztchangeostraC13[2]) * zalphaostraC13[1] + 
       (zt[i] >= zt_min) * (zt[i] < ztchangeostraC13[2]) * zbetaostraC13[1]  * XostraC13_1[i] + 
      
      # Segment 2: 
        (zt[i] >= ztchangeostraC13[2]) * zalphaostraC13[2] + 
        (zt[i] >= ztchangeostraC13[2]) * zbetaostraC13[2] * XostraC13_2[i]
    
    XTOC_1[i] = min(zagemodel[i], ztchangeTOC[1])
    XTOC_2[i] = min(zagemodel[i], ztchangeTOC[2]) - ztchangeTOC[1]
    XTOC_3[i] = min(zagemodel[i], zt_max) - ztchangeTOC[2]


    zyTOC[i]  ~  dt(zmuTOC[i],  1/zsigmaTOC[i]^2, nuTOC) 
    zmuTOC[i] <-       
      # Segment 1: 
        (zt[i] >= zt_min) * (zt[i] < ztchangeTOC[1]) * zalphaTOC[1] + 
    
      # Segment 2: 
        (zt[i] >= ztchangeTOC[1]) * (zt[i] < ztchangeTOC[2]) * zalphaTOC[2] + 
    
      # Segment 3: 
        (zt[i] >= ztchangeTOC[2]) * zalphaTOC[3] + 
        (zt[i] >= ztchangeTOC[2]) * zbetaTOC[3] * XTOC_3[i]
    
    zagemodel[i] ~ dnorm(zt[i], 1/zterr[i]^2)

    zsigmaforamO18[i] = max(0, 
      (zt[i] >= zt_min) * zsigmaforamO181 )

    zsigmaforamC13[i] = max(0, 
      (zt[i] >= zt_min) * zsigmaforamC131 )
    
    zsigmaostraO18[i] = max(0, 
      (zt[i] >= zt_min) * zsigmaostraO181 )
    
    zsigmaostraC13[i] = max(0, 
      (zt[i] >= zt_min) * zsigmaostraC131 )

    zsigmaTOC[i] = max(0, 
      (zt[i] >= zt_min) * zsigmaTOC1 )  
  }

      ztchangeforamO18.temp[1] ~ dt( zt.change[1] ,
                                 1/zcp_priorsd^2 , 4)T(zt_min, zt_max)   
      ztchangeforamO18.temp[2] ~ dt( zt.change[2] , 
                                 1/zcp_priorsd^2 , 4)T(ztchangeforamO18.temp[1], zt_max) 
      ztchangeforamO18[1:2] <- sort(ztchangeforamO18.temp) #CPs need to be ordered

      ztchangeforamC13.temp[1] ~ dt( zt.change[1] , 
                                 1/zcp_priorsd^2 , 4)T(zt_min, zt_max)   
      ztchangeforamC13.temp[2] ~ dt( zt.change[2] ,
                                 1/zcp_priorsd^2 , 4)T(ztchangeforamC13.temp[1], zt_max) 
      ztchangeforamC13[1:2] <- sort(ztchangeforamC13.temp) #CPs need to be ordered

      ztchangeostraO18.temp[1] ~ dt( zt.change[1] ,
                                 1/zcp_priorsd^2 , 4)T(zt_min, zt_max)   
      ztchangeostraO18.temp[2] ~ dt( zt.change[2] ,
                                 1/zcp_priorsd^2 , 4)T(ztchangeostraO18.temp[1], zt_max) 
      ztchangeostraO18[1:2] <- sort(ztchangeostraO18.temp) #CPs need to be ordered

      ztchangeostraC13.temp[1] ~ dt( zt.change[1] ,
                                 1/zcp_priorsd^2 , 4)T(zt_min, zt_max)   
      ztchangeostraC13.temp[2] ~ dt( zt.change[2] ,
                                 1/zcp_priorsd^2 , 4)T(ztchangeostraC13.temp[1], zt_max) 
      ztchangeostraC13[1:2] <- sort(ztchangeostraC13.temp) #CPs need to be ordered

      ztchangeTOC.temp[1] ~ dt( zt.change[1] ,
                               1/zcp_priorsd^2 , 4)T(zt_min, zt_max)   
      ztchangeTOC.temp[2] ~ dt( zt.change[2] , 
                               1/zcp_priorsd^2 , 4)T(ztchangeTOC.temp[1], zt_max) 
      ztchangeTOC[1:2] <- sort(ztchangeTOC.temp) #CPs need to be ordered

      zt.change.temp[1] ~ dt(zcp1_priormean, 1/zcp_priorsd^2, 4)T(zt_min, zt_max) 
      zt.change.temp[2] ~ dt(zcp2_priormean, 1/zcp_priorsd^2, 4)T(zt_min, zt_max) 
      zt.change[1:2] <- sort(zt.change.temp) #CPs need to be ordered

  # Priors
  for ( j in 1:3 ) {
    zalphaforamO18[j] ~ dt(0.0, 1/2^2, 10) 
    zbetaforamO18[j]  ~ dt(0.0, 1/2^2, 10)
    
    zalphaforamC13[j] ~ dt(0.0, 1/2^2, 10) 
    zbetaforamC13[j]  ~ dt(0.0, 1/2^2, 10) 
    
    zalphaostraO18[j] ~ dt(0.0, 1/2^2, 10) 
    zbetaostraO18[j]  ~ dt(0.0, 1/2^2, 10) 
    
    zalphaostraC13[j] ~ dt(0.0, 1/2^2, 10) 
    zbetaostraC13[j]  ~ dt(0.0, 1/2^2, 10) 
    
    zalphaTOC[j] ~ dt(0.0, 1/2^2, 10) 
    zbetaTOC[j]  ~ dt(0.0, 1/2^2, 10)    
  }
  
    zsigmaforamO181 ~ dnorm(0, 1/1^2) T(0, )
    zsigmaforamC131 ~ dnorm(0, 1/1^2) T(0, )
    zsigmaostraO181 ~ dnorm(0, 1/1^2) T(0, )
    zsigmaostraC131 ~ dnorm(0, 1/1^2) T(0, )
    zsigmaTOC1 ~ dnorm(0, 1/1^2) T(0, )
    
    nuforamO18 ~ dexp(1/30.0)
    nuforamC13 ~ dexp(1/30.0)
    nuostraO18 ~ dexp(1/30.0)
    nuostraC13 ~ dexp(1/30.0)
    nuTOC ~ dexp(1/30.0)
      
  # Transform to original scale:
    
    for ( m in 1:3 ) {
      alphaforamO18[m] <- zalphaforamO18[m] * yforamO18sd + yforamO18m - 
                        zbetaforamO18[m] * yforamO18sd / tsd * tm
      betaforamO18[m] <- zbetaforamO18[m] * yforamO18sd / tsd 
      
      alphaforamC13[m] <- zalphaforamC13[m] * yforamC13sd + yforamC13m -
                        zbetaforamC13[m] * yforamC13sd / tsd * tm
      betaforamC13[m] <- zbetaforamC13[m] * yforamC13sd / tsd 
      
      alphaostraO18[m] <- zalphaostraO18[m] * yostraO18sd + yostraO18m -
                        zbetaostraO18[m] * yostraO18sd / tsd * tm
      betaostraO18[m] <- zbetaostraO18[m] * yostraO18sd / tsd 
      
      alphaostraC13[m] <- zalphaostraC13[m] * yostraC13sd + yostraC13m -
                        zbetaostraC13[m] * yostraC13sd / tsd * tm
      betaostraC13[m] <- zbetaostraC13[m] * yostraC13sd / tsd 
      
      alphaTOC[m] <- zalphaTOC[m] * yTOCsd + yTOCm -
                    zbetaTOC[m] * yTOCsd / tsd * tm
      betaTOC[m] <- zbetaTOC[m] * yTOCsd / tsd 
    }
    
    for ( k in 1:2 ){
    t.change[k] <- zt.change[k] * tsd  +tm
    tchangeforamO18[k] <- ztchangeforamO18[k] * tsd  +tm
    tchangeforamC13[k] <- ztchangeforamC13[k] * tsd  +tm
    tchangeostraO18[k] <- ztchangeostraO18[k] * tsd  +tm
    tchangeostraC13[k] <- ztchangeostraC13[k] * tsd  +tm
    tchangeTOC[k] <- ztchangeTOC[k] * tsd  +tm
    }
    #sigmaforamO18  <- zsigmaforamO18  * yforamO18sd
    #sigmaforamC13  <- zsigmaforamC13  * yforamC13sd
    #sigmaostraO18  <- zsigmaostraO18  * yostraO18sd
    #sigmaostraC13  <- zsigmaostraC13  * yostraC13sd
    #sigmaTOC  <- zsigmaTOC  * yTOCsd
    } " 

  # Write out modelString to a text file
  writeLines( model , con="TEMPmodel.txt" )
  
  model_run <- run.jags(model ="TEMPmodel.txt",
                method="parallel",
                data = datalist,
                monitor=jags_pars,
                inits=NA , 
                n.chains=nChains ,
                adapt=adaptSteps ,
                burnin=burnInSteps , 
                sample=ceiling(numSavedSteps/nChains) ,
                thin=thinSteps ,
                summarise=FALSE ,
                plots=FALSE,
                modules='glm' )
  
  my_run <- model_run
  return( my_run )
}

In [ ]:
smryMCMC = function(  codaSamples , 
                      saveName=NULL ) {
  mcmcMat = as.matrix(codaSamples,chains=FALSE)
  paramNames = colnames(mcmcMat)
  summaryInfo = NULL
  for ( pName in paramNames ) {
    summaryInfo = rbind( summaryInfo ,  summarizePost( mcmcMat[,pName] ) )
  }
  rownames(summaryInfo) = paramNames
  if ( !is.null(saveName) ) {
    write.csv( summaryInfo , file=paste(saveName,"_SummaryInfo_cp",".csv",sep="") )
  }
  return( summaryInfo )
}


# Loading and tidying the data

In [ ]:
bafa_data_url_text <- paste("https://raw.githubusercontent.com/zboraon/",
                            "LakeBafeClosure_Bayesian_cps/main/",
                            "data/isotope_tictoc.csv", sep = "")
bafa_data_url <- getURL(bafa_data_url_text)
bafa_data <- read.csv(text = bafa_data_url)

bafa_age_url_text <- paste("https://raw.githubusercontent.com/zboraon/",
                  "LakeBafeClosure_Bayesian_cps/main/",
                  "data/bafa_age_depth.csv", sep = "")
bafa_age_url <- getURL(bafa_age_url_text)
bafa_age <- read.csv(text = bafa_age_url)

# Tidy data
bafa_data_interpolated <- bafa_data %>%
  mutate(ostra_o18 = na.approx(ostra_o18))

bafa_data_interpolated <- bafa_data_interpolated %>%
  mutate(ostra_c13 = na.approx(ostra_c13))

bafa_data_interpolated <- bafa_data_interpolated[,!(names(bafa_data_interpolated) 
                            %in% c("ostra_depth"))]

bafa_data_interpolated <- tail(bafa_data_interpolated,-6)

# assign data to vectors
yforamO18 <- bafa_data_interpolated$foram_o18
yforamC13 <- bafa_data_interpolated$foram_c13
yostraO18 <- bafa_data_interpolated$ostra_o18
yostraC13 <- bafa_data_interpolated$ostra_c13
yTOC      <- bafa_data_interpolated$TOC

depth <- bafa_data_interpolated$depth
L <- length(bafa_data_interpolated$depth)

agemean <- interp1(bafa_age$depth, bafa_age$mean_yrsBP, depth, "linear", extrap = TRUE)
ageerror <- interp1(bafa_age$depth, bafa_age$sd_age, depth, "linear", extrap = TRUE)

# collect them to a list
datalist <- list(yforamO18 = yforamO18,
                 yforamC13 = yforamC13,
                 yostraO18 = yostraO18,
                 yostraC13 = yostraC13,
                 yTOC = yTOC,
                 L = L,
                 agemean =agemean,
                 ageerror=ageerror/2 # dividing to two, because the data is 2 sigma error
                 ) 

# Run the loop

In [ ]:
sampleNO <- 10000 
thinstepsNO <- 20
adaptNO <- 10000 
burninNO <- 10000 
chainNO <- 4

# run the model 
my_run <- changepnt_lakebafa(testeddata = datalist,
                                 #inits = initsList, 
                                 sample = sampleNO, adaptSteps = adaptNO, 
                                 burnInSteps = burninNO, nChains = chainNO,  
                                 thinSteps = thinstepsNO
                                 )


  save(my_run, file="LakeBafa2cps.Rdata" )

In [ ]:
      codaSamples <- as.mcmc.list(my_run)
      summaryInfo = smryMCMC(codaSamples, saveName="LakeBafe_2cps")